In [4]:
!pip install pymorphy3
!pip uninstall docx
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.0 MB/s eta 0:00:00


In [13]:
import re
import pymorphy3
from docx import Document

def check_keywords(text):
    morph = pymorphy3.MorphAnalyzer()
    words = text.split()

    state_registration_base = [morph.parse('государственная')[0].normal_form,
                               morph.parse('регистрация')[0].normal_form]
    license_base = [morph.parse('лицензионное')[0].normal_form,
                               morph.parse('соглашение')[0].normal_form,
                               morph.parse('договор')[0].normal_form,
                               morph.parse('лицензиар')[0].normal_form,
                               morph.parse('лицензиат')[0].normal_form]
    rospatent_base = morph.parse('роспатент')[0].normal_form
    working_days_base = [morph.parse('рабочие')[0].normal_form,
                         morph.parse('дни')[0].normal_form]
    competition_base = [morph.parse('запретить')[0].normal_form,
                         morph.parse('запрещать')[0].normal_form,
                         morph.parse('конкуренция')[0].normal_form]
    automatic_null_base = [morph.parse('автоматический')[0].normal_form,
                         morph.parse('прекращение')[0].normal_form,
                         morph.parse('расторжение')[0].normal_form]

    state_registration_found = False
    rospatent_found = False
    working_days_found = False
    license_found = False
    competition_found = False
    automatic_null_found = False

    registration_positions = []
    working_days_positions = []

    comp_ind = -100000

    for index in range(len(words) - 1):
        first_word = morph.parse(words[index])[0].normal_form
        second_word = morph.parse(words[index + 1])[0].normal_form

        if first_word == state_registration_base[0] and second_word == state_registration_base[1]:
            state_registration_found = True
            registration_positions.append(index)
        if first_word == license_base[0] and (second_word == license_base[1] or second_word == license_base[2]):
            license_found = True
        if not license_found:
            if license_base[3] in morph.parse(words[index])[0].normal_form or license_base[4] in morph.parse(words[index])[0].normal_form:
                license_found = True
        if rospatent_base in morph.parse(words[index])[0].normal_form:
            rospatent_found = True
        if competition_base[0] in morph.parse(words[index])[0].normal_form or competition_base[1] in morph.parse(words[index])[0].normal_form:
            comp_ind = index
        if competition_base[2] in morph.parse(words[index])[0].normal_form and index - comp_ind <= 4:
            competition_found = True
        if first_word == working_days_base[0] and second_word == working_days_base[1]:
            working_days_found = True
            working_days_positions.append(index)
        if first_word == automatic_null_base[0] and (second_word == automatic_null_base[1] or second_word == automatic_null_base[2]):
            automatic_null_found_found = True

    results = {
        "государственная регистрация": state_registration_found,
        "роспатент": rospatent_found,
        "лицензионное соглашение": license_found,
        "рабочие дни": working_days_found,
        "запрет конкуренции": competition_found,
        "автоматическое расторжение": automatic_null_found
    }
    print(results)
    for pos in registration_positions:
        start = max(0, pos - 40)
        end = min(len(words), pos + 2 + 40)
        substring = ' '.join(words[start:end])
        pattern = r'(d+)s+рабочихs+дней'
        matches = re.findall(pattern, substring)



    for pos in working_days_positions:
        start = max(0, pos - 40)
        end = min(len(words), pos + 2 + 40)
        substring = ' '.join(words[start:end])
        pattern = r'(d+)s+рабочиеs+дни'
        matches = re.findall(pattern, substring)

        if matches:
            results["рабочие дни"].extend(matches)

    return results

if __name__ == "__main__":

    source = Document("contract.docx")
    document_text = ''
    paras = source.paragraphs
    for para in paras:
        document_text += f'{para.text}\n'

    results = check_keywords(document_text)

    current_mark = 3

    if results['лицензионное соглашение']:
        current_mark = 1
    if results['автоматическое расторжение']:
        current_mark = 1
    if results['запрет конкуренции']:
        current_mark = 1
    if not results['государственная регистрация'] and not results['роспатент']:
        current_mark = 1
    elif not results['рабочие дни']:
        current_mark = 2

    print(f'Юридическая оценка договора: {current_mark}')




{'государственная регистрация': True, 'роспатент': True, 'лицензионное соглашение': False, 'рабочие дни': True, 'запрет конкуренции': False, 'автоматическое расторжение': False}
Юридическая оценка договора: 3
